In [367]:
import os
import sklearn
import pandas as pd
import csv

In [368]:
n_gram_value = 5

In [369]:
def get_ngrams(input_list, n):    
    return zip(*[input_list[i:] for i in range(n)])


In [370]:
def getAllPossibleWords(file_path): 
    possible_words=[]
    with open(file_path) as f:
        for line in f:
            for word in line.split():
                possible_words.append(word)     
   # possible_words = map(lambda x : unidecode.unidecode(x),possible_words)
    for c in ['`',"[","]","(",")","\"",".","''",","]:
        possible_words = map(lambda x: x.replace(c,""),possible_words)
        
    possible_words = map(lambda x: x.replace('&','And'),possible_words)
    #possible_words = map(lambda x: x.replace('of','Of'),possible_words)
    possible_words=filter(lambda x:len(x)>0,possible_words)    
    
    return possible_words

In [385]:
def getListFromFiles(filename):
    file_path = os.path.join(os.getcwd(),filename)
    info_list = []
    with open(file_path) as f:
        for line in f:
            for word in line.split():
                info_list.append(word)
    return info_list

In [386]:
def writeToCsv(processed_list):
    with open("output-sreejita.csv",'wb') as resultFile:
        wr = csv.writer(resultFile, dialect='excel')
        #for row in processed_list:
        wr.writerows(processed_list)

In [387]:
'''def preprocessing(possible_words,doc_id):    
    suffix_list = getListFromFiles("company-suffixes.txt")
    prefix_list = getListFromFiles("company-prefixes.txt")    
    common_words = getListFromFiles("common-words.txt")
    preprocessed_list =[]    
    marked_list=[]
    word_number=len(possible_words)
    for i in range(5,0,-1):
        word_count=0
        ngram_list = get_ngrams(possible_words,i)
        for ngram in ngram_list:            
            #print ngram   
            
            #if markup is both at the beginning or at the end of the word group accept and label as 1
            if(len(ngram)>1 and '<markup>' in ngram[0] and '</markup>' in ngram[len(ngram)-1] and
              all((word[0].isupper() or "markup" in word) for word in ngram) and 
              ["<markup>" in word for word in ngram].count(True)==1):
                preprocessed_list.append([' '.join(ngram),doc_id,word_count,word_count+i-1,1])
            
            #get all instances of format <markup>Microsoft</markup>
            if(len(ngram)==1 and '</markup>' in ngram[0] and '<markup>' in ngram[0]):
                preprocessed_list.append([' '.join(ngram),doc_id,word_count,word_count+i-1,1])            
                    
            else:
                #prune away all n grams with non uppercase first character
                if(all(word[0].isupper() and "markup" not in word for word in ngram)):       
                    
                    # prune common words
                    if(any(word not in common_words for word in ngram)):                        
                        preprocessed_list.append([' '.join(ngram),doc_id,word_count,word_count+i-1,0])

                    #prune unigrams 
                    elif(i==1 and word_number> word_count+1):                        
                        
                        #add the unigram to the list if the next word begins with lower case
                        if(possible_words[word_count+1][0].islower()):
                            preprocessed_list.append([' '.join(ngram),doc_id,word_count,word_count+i-1,0])  
            word_count = word_count+1
    return preprocessed_list   ''' 

'def preprocessing(possible_words,doc_id):    \n    suffix_list = getListFromFiles("company-suffixes.txt")\n    prefix_list = getListFromFiles("company-prefixes.txt")    \n    common_words = getListFromFiles("common-words.txt")\n    preprocessed_list =[]    \n    marked_list=[]\n    word_number=len(possible_words)\n    for i in range(5,0,-1):\n        word_count=0\n        ngram_list = get_ngrams(possible_words,i)\n        for ngram in ngram_list:            \n            #print ngram   \n            \n            #if markup is both at the beginning or at the end of the word group accept and label as 1\n            if(len(ngram)>1 and \'<markup>\' in ngram[0] and \'</markup>\' in ngram[len(ngram)-1] and\n              all((word[0].isupper() or "markup" in word) for word in ngram) and \n              ["<markup>" in word for word in ngram].count(True)==1):\n                preprocessed_list.append([\' \'.join(ngram),doc_id,word_count,word_count+i-1,1])\n            \n            #get all

In [388]:
def preprocessing(possible_words,doc_id):    
    suffix_list = getListFromFiles("company-suffixes.txt")
    prefix_list = getListFromFiles("company-prefixes.txt")    
    common_words = getListFromFiles("common-words.txt")
    preprocessed_list =[]    
    marked_list=[]
    word_number=len(possible_words)
    for i in range(5,0,-1):
        word_count=0
        ngram_list = get_ngrams(possible_words,i)
        for ngram in ngram_list:            
            #print ngram   
            
            #if markup is both at the beginning or at the end of the word group accept and label as 1
            if(len(ngram)>1 and '<markup>' in ngram[0] and '</markup>' in ngram[len(ngram)-1] and
              all((word[0].isupper() or "markup" in word) for word in ngram) and 
              ["<markup>" in word for word in ngram].count(True)==1): 
                company_tuple = ngram
                for string in ["<markup>","</markup>"]:
                    company_tuple = map(lambda x : x.replace(string,""),company_tuple)                
                preprocessed_list.append([' '.join(company_tuple),doc_id,word_count,word_count+i-1,1])
            
            #get all instances of format <markup>Microsoft</markup>
            elif(len(ngram)==1 and '</markup>' in ngram[0] and '<markup>' in ngram[0]):
                
                company_tuple = ngram
                for string in ["<markup>","</markup>"]:
                    company_tuple = map(lambda x : x.replace(string,""),company_tuple) 
                
                preprocessed_list.append([' '.join(company_tuple),doc_id,word_count,word_count+i-1,1])            
                    
            else:
                #prune away all n grams with non uppercase first character
                if(all(word[0].isupper() and "markup" not in word for word in ngram)):  
                    
                    #prune unigrams 
                    if(i==1):                        
                        if(ngram[0] not in suffix_list):
                            if(word_number> word_count+1 and ngram[0] not in common_words):
                                #add the unigram to the list if the next word begins with lower case
                                if(possible_words[word_count+1][0].islower()):
                                    preprocessed_list.append([' '.join(ngram),doc_id,word_count,word_count+i-1,0])
                    
                    #prune common words
                    elif(any(word not in common_words for word in ngram)):                        
                        preprocessed_list.append([' '.join(ngram),doc_id,word_count,word_count+i-1,0])                   

                    
                     
            word_count = word_count+1
    return preprocessed_list    

In [389]:
def generateFeatures(filename, processed_list,possible_words):
    
    company_suffixes = getListFromFiles("company-suffixes.txt")
    company_prefixes = getListFromFiles("company-prefixes.txt")    
    company_list=[]
    final_processed_list = []
    splittedFilename = filename.split("-")
    i=0
    print "possible_words- "  
    print possible_words
    for company in processed_list:
        i+=1
        #Feature 1 : name has any prefix or suffix of a company 
        #if company name has any one of the suffixes or prefixes set it to true 
        hasCompanyid=0        
        if(company[0].split()[-1] in company_suffixes or company[0].split()[0] in company_prefixes):
            hasCompanyid = 1
            company_list.append(company[0])            
        #company.append(hasCompanyid)
        
        
        #has Company name at the beginning ?
        #Feature 2: name occurs at the beginning of a company name
        hasCompanyNameFirst = 0
        l1= [x for x in company[0].split() if x not in company_prefixes]
        l2=[]
        for comp in company_list:    
            if(comp.split()[0] in company_prefixes):
                l2.append(" ".join(comp.split()[1:]))
            else:
                l2.append(comp)
        if(any(word in l2 for word in l1)):
            hasCompanyNameFirst = 1
        elif(any(company[0].split()[0]== word.split()[0] for word in company_list)):
            hasCompanyNameFirst = 1 
        
            
        
        #has Company name as a substring ?
        #Feature 3: name is a substring in the company name
        hasNameSubstring=0
        for comp in company_list:
            l1 = company[0].split()
            l2 = comp.split()
            if(len(set(l1)&set(l2)) > 0 ):
                hasNameSubstring=1       
       
         
        
        #Feature 4: name in filename
        isInFilename = 0
        splittedInstance = company[0].split()
        for sp in splittedInstance:
            if sp.lower() in splittedFilename:
                isInFilename = 1 
        
        
        #Feature 5-y - Check neibhouring words - check 3 words before and 3 words after
        starting_loc = company[2]
        ending_loc = company[3]
        words_before_starting_loc = []
        words_after_ending_loc = []
        j= starting_loc-1
        count = 1
        while j >= 0 and count <= 3:
            words_before_starting_loc.append(possible_words[company[2]-count])
            j-=1
            count+=1
        
        j = ending_loc+1
        count = 1
        while j < len(possible_words) and count <= 3:
            words_after_ending_loc.append(possible_words[company[3]+count])
            j+=1
            count+=1
        
        #Feature 5 - Check for word "based" before the company
        has_based = 0
        for word in words_before_starting_loc:
            if "based" in word.lower():
                has_based = 1
        
        #Feature 6 "'s" is after the company
        has_apostrophe_s = 0
        for word in words_after_ending_loc:
            if "'s" in word.lower():
                has_apostrophe_s = 1
        '''
        #Feature 7 "," is after/before the company
        has_comma = 0
        for word in words_after_ending_loc:
            if "," in word.lower():
                has_comma = 1
        
        for word in words_before_starting_loc:
            if "," in word.lower():
                has_comma = 1
        '''
        #Feature 8 designations before/after company
        desig = open('designations.txt').read().split('\n')
        desig_lower = [x.lower() for x in desig]
        has_desig = 0
        for word in words_after_ending_loc:
            if word.lower() in desig_lower:
                has_desig = 1
              
        for word in words_before_starting_loc:
            if word.lower() in desig_lower:
                has_desig = 1 
         
        #Feature 9 "said/told" in the before company
        has_said = 0
        for word in words_before_starting_loc:
            if "said" in word.lower() or "told" in word.lower():
                has_said = 1
        
        '''
        #Feature 10 Prune away Examples if previous/next word starts with a Capital
        surrounded_capital = 0
        if len(words_before_starting_loc) > 0:
            prev_word = words_before_starting_loc[0]
            if len(prev_word) > 0 and prev_word[0].isupper():
                surrounded_capital = 1
        if len( words_after_ending_loc) > 0:
            next_word = words_after_ending_loc[0]
            if len(next_word) > 0 and next_word[0].isupper():
                surrounded_capital = 1
        '''
          
                
        
        print "company:- " + company[0]
        print "words_after_ending_loc"
        print words_after_ending_loc
        print "words_before_starting_loc"
        print words_before_starting_loc
        
        company.extend([hasCompanyid, hasCompanyNameFirst, hasNameSubstring, isInFilename, has_based, has_apostrophe_s, has_desig, has_said])
        
            

In [401]:
base_directory = os.path.dirname(os.getcwd())
data_directory = os.path.join(base_directory,"Dataset","Training_1")
file_list = os.listdir(data_directory)
possible_words=[]
processed_list=[]
count=0
#doc_id = 101
for file in file_list:
    relevant_filename = file[4:-4]
    doc_id = file[0:3]
    #print(doc_id)
    #if(int(doc_id)!=95):
    #    continue    
    possible_words = getAllPossibleWords(os.path.join(data_directory,file))     
    
    #start from here .. processessed list has the structure[[word,doc_id,word_count_start,word_count_end,label]]
    ''''processed_list.append(preprocessing(possible_words,doc_id))
    writeToCsv(processed_list)'''
    processed_list_for_doc = preprocessing(possible_words,doc_id)
    #print processed_list_for_doc
    generateFeatures(relevant_filename, processed_list_for_doc,possible_words)
    processed_list.extend(processed_list_for_doc)
    #writeToCsv(processed_list) 
  

possible_words- 
['American', 'Idol', 'the', 'most', 'popular', 'show', 'in', 'primetime', 'television', 'withstood', 'last', "week's", 'criticism', 'over', 'judging', 'and', 'returned', 'last', 'night', 'with', 'the', 'third-largest', 'audience', 'for', 'an', 'entertainment', 'show', 'this', 'season', 'With', 'about', '326', 'million', 'viewers', 'according', 'to', 'overnight', 'results', 'from', 'Nielsen', 'Media', 'Research', 'the', 'latest', 'Idol', 'trailed', 'only', 'the', 'two', 'Idol', 'shows', 'that', 'opened', 'its', 'sixth', 'season', 'on', 'Fox', 'last', 'week', 'with', 'audiences', 'of', '374', 'million', 'and', '37', 'million', 'and', 'the', 'National', 'Football', "League's", 'American', 'Conference', 'championship', 'game', 'on', 'CBS', 'last', 'Sunday', 'which', 'drew', '435', 'million', 'The', 'first', 'two', 'Idol', 'shows', 'were', 'the', 'second', 'and', 'third', 'most-watched', 'of', 'the', 'series', 'since', 'the', "show's", '2002', 'debut', 'They', 'also', 'spur

company:- July
words_after_ending_loc
['award', 'Jazz', 'the']
words_before_starting_loc
['a', 'in', 'period']
company:- Jazz
words_after_ending_loc
['the', 'regional', 'airline']
words_before_starting_loc
['award', 'July', 'a']
company:- ACE
words_after_ending_loc
['must', 'pay', 'the']
words_before_starting_loc
['by', 'owned', 'percent']
company:- Jazz
words_after_ending_loc
['said', 'today', 'in']
words_before_starting_loc
['Halifax-based', '2008', 'July']
company:- Picher
words_after_ending_loc
['matching', 'those', 'he']
words_before_starting_loc
['Michel', 'arbitrator', 'by']
company:- Jazz
words_after_ending_loc
['employees', 'in', 'July']
words_before_starting_loc
['other', '1660', 'to']
company:- ACE
words_after_ending_loc
['is', 'awaiting', 'an']
words_before_starting_loc
['July', 'in', 'employees']
company:- Union
words_after_ending_loc
['of', 'Public', 'Employees']
words_before_starting_loc
['Canadian', 'the', 'with']
company:- Employees
words_after_ending_loc
['which', 're

possible_words- 
['<markup>OAO', 'Aeroflot</markup>', 'AFLT', 'eastern', 'Europe', "'s", 'largest', 'airline', 'carried', '87', 'percent', 'more', 'passengers', 'last', 'year', 'as', 'the', "country's", 'growing', 'economy', 'boosted', 'travel', 'State-controlled', '<markup>Aeroflot</markup>', 'flew', '73', 'million', 'passengers', 'in', '2006', 'the', 'Moscow-based', 'airline', 'said', 'today', 'in', 'a', 'statement', 'The', 'company', 'handled', '02', 'percent', 'less', 'cargo', 'or', '145300', 'tons', 'last', 'year', 'The', 'load', 'factor', 'or', 'proportion', 'of', 'seats', 'filled', 'increased', 'to', '701', 'percent', 'from', '691', 'percent', 'in', '2005', '<markup>Aeroflot</markup>', 'is', 'attracting', 'more', 'travelers', 'as', 'Russia', "'s", 'economy', 'expands', 'for', 'a', 'ninth', 'consecutive', 'year', 'backed', 'by', 'rising', 'oil', 'prices', 'The', 'carrier', 'joined', 'the', 'SkyTeam', 'alliance', 'of', '<markup>Air', 'France-KLM', 'Group</markup>', 'and', '<markup

['of', 'maximizing', 'its']
words_before_starting_loc
['of', 'District', 'the']
company:- Act
words_after_ending_loc
['and', 'a', 'parallel']
words_before_starting_loc
['Claims', 'False', 'federal']
company:- We
words_after_ending_loc
['respectfully', 'but', 'strongly']
words_before_starting_loc
['trial', 'three-week', 'a']
company:- Amerigroup
words_after_ending_loc
['spokesman', 'Kent', 'Jenkins']
words_before_starting_loc
['appeal', 'will', 'we']
company:- Jenkins
words_after_ending_loc
['said', 'outside', 'the']
words_before_starting_loc
['Kent', 'spokesman', '<markup>Amerigroup</markup>']
company:- August
words_after_ending_loc
['in', 'a', 'contract']
words_before_starting_loc
['in', 'came', 'million']
company:- This
words_after_ending_loc
['is', 'not', 'a']
words_before_starting_loc
['Louisiana', 'in', 'plants']
company:- Burns
words_after_ending_loc
['spokesman', 'for', 'Taxpayers']
words_before_starting_loc
['Patrick', 'said', 'trial']
company:- Fraud
words_after_ending_loc
['a

possible_words- 
['<markup>Anglo', 'Irish', 'Bank', 'Plc</markup>', "Ireland's", 'third-largest', 'bank', 'said', 'fiscal', 'first-half', 'earnings', 'will', 'rise', 'close', 'to', '40', 'percent', 'helped', 'by', 'demand', 'for', 'loans', 'Net', 'loans', 'increased', '73', 'billion', 'euros', '$96', 'billion', 'in', 'the', 'five', 'months', 'through', 'February', 'according', 'to', 'a', 'statement', 'by', 'the', 'Dublin-based', 'company', 'which', 'had', 'earnings', 'per', 'share', 'of', '413', 'cents', 'a', 'share', 'a', 'year', 'earlier', 'The', 'first', 'half', 'tees', 'us', 'up', 'for', 'a', 'very', 'strong', 'financial', 'outturn', 'in', '2007', 'Chief', 'Financial', 'Officer', 'Matt', 'Moran', 'said', 'in', 'a', 'telephone', 'interview', 'today', 'Chief', 'Executive', 'Officer', 'David', 'Drumm', 'is', 'focusing', 'on', 'business', 'lending', 'as', 'the', 'bank', 'competes', 'with', 'larger', 'rivals', '<markup>Bank', 'of', 'Ireland', 'Plc</markup>', 'and', '<markup>Allied', 'Ir

...

company:- Severstal
words_after_ending_loc
['dropped', '1230', 'rubles']
words_before_starting_loc
['of', 'CHMF', 'Shares']
company:- Micex
words_after_ending_loc
['exchange', 'in', 'Moscow']
words_before_starting_loc
['the', 'on', 'rubles']
company:- Moscow
words_after_ending_loc
['valuing', 'the', 'company']
words_before_starting_loc
['in', 'exchange', 'Micex']
possible_words- 
['Officials', 'at', '<markup>Shougang', 'Hierro', 'Peru', 'SAA</markup>', "Peru's", 'only', 'iron', 'mining', 'company', 'were', 'poised', 'to', 'resume', 'talks', 'with', 'workers', 'today', 'to', 'end', 'an', 'eight-day', 'strike', 'a', 'union', 'official', 'said', 'Subcontracted', 'workers', 'at', 'the', '<markup>Shougang</markup>', 'mine', 'on', 'the', 'south', 'coast', 'of', 'Peru', 'are', 'one', 'of', 'three', 'groups', 'that', 'have', 'yet', 'to', 'negotiate', 'labor', 'settlements', 'since', 'a', 'national', "miners'", 'strike', 'began', 'April', '30', "Peru's", 'Mining', 'Federation', 'which', 'repres

words_after_ending_loc
['Trends', 'released', 'today']
words_before_starting_loc
['of', 'edition', '2007']
company:- Industry Trends
words_after_ending_loc
['released', 'today', 'Publishers']
words_before_starting_loc
['Book', 'of', 'edition']
company:- Scholastic Corp
words_after_ending_loc
['SCHL', "'s", 'Harry']
words_before_starting_loc
['by', 'year', 'this']
company:- Harry Potter
words_after_ending_loc
['and', 'the', 'Deathly']
words_before_starting_loc
["'s", 'SCHL', 'Corp</markup>']
company:- Deathly Hallows
words_after_ending_loc
['the', 'seventh', 'and']
words_before_starting_loc
['the', 'and', 'Potter']
company:- JK Rowling
words_after_ending_loc
['a', 'billionaire', 'Book']
words_before_starting_loc
['author', 'made', 'and']
company:- Edward Atorino
words_after_ending_loc
['a', 'media', 'analyst']
words_before_starting_loc
['said', 'dramatically', 'not']
company:- Benchmark Co
words_after_ending_loc
['in', 'New', 'York']
words_before_starting_loc
['with', 'analyst', 'media'

['sides', 'both', 'consider']
company:- Bancrofts
words_after_ending_loc
['since', 'Murdoch', 'announced']
words_before_starting_loc
['the', 'with', 'meeting']
company:- Murdoch
words_after_ending_loc
['announced', 'his', 'takeover']
words_before_starting_loc
['since', 'Bancrofts', 'the']
company:- Journal
words_after_ending_loc
['on', 'May', '1']
words_before_starting_loc
['Street', 'Wall', 'the']
company:- Bancroft
words_after_ending_loc
['family', 'members', 'with']
words_before_starting_loc
['1', 'May', 'on']
company:- NWSA
words_after_ending_loc
['has', 'offered', '<markup>Dow']
words_before_starting_loc
['Corp</markup>', '<markup>News', 'offers']
company:- Murdoch's
words_after_ending_loc
['offer', 'became', 'public']
words_before_starting_loc
['before', 'day', 'the']
company:- Bancrofts
words_after_ending_loc
['are', 'heirs', 'to']
words_before_starting_loc
['The', 'public', 'became']
company:- Shares
words_after_ending_loc
['of', '<markup>Dow', 'Jones</markup>']
words_before_st

possible_words- 
['<markup>Vedanta', 'Resources', 'Plc</markup>', 'VED', 'agreed', 'to', 'buy', '<markup>Mitsui', 'And', 'Co</markup>', "'s", 'entire', 'stake', 'in', 'Indian', 'iron-ore', 'exporter', '<markup>Sesa', 'Goa', 'Ltd</markup>', 'SESA', 'for', '$981', 'million', 'defeating', 'rivals', 'including', 'Arcelor', 'Mittal', 'to', 'secure', 'supplies', 'of', 'the', 'steel-making', 'raw', 'material', '<markup>Vedanta</markup>', 'the', 'biggest', 'producer', 'of', 'copper', 'and', 'zinc', 'in', 'India', 'will', 'pay', '2036', 'rupees', '$49', 'per', 'share', 'for', 'a', '51', 'percent', 'stake', '17', 'percent', 'more', 'than', "yesterday's", 'closing', 'price', '<markup>Vedanta</markup>', 'will', 'offer', 'to', 'buy', 'a', 'further', '20', 'percent', 'at', 'the', 'same', 'price', '<markup>Adviser', 'Nomura', 'Holdings', 'Inc</markup>', 'will', 'lend', '$11', 'billion', 'to', 'fund', 'the', 'takeover', 'Chairman', 'Anil', 'Agarwal', 'beat', 'fellow', 'Indian', 'billionaires', 'Lakshm

company:- North America
words_after_ending_loc
['and', 'Europe', 'and']
words_before_starting_loc
['in', 'mills', 'to']
company:- US Steel
words_after_ending_loc
['spokesman', 'John', 'Armstrong']
words_before_starting_loc
['Minnesota', 'in', 'mines']
company:- John Armstrong
words_after_ending_loc
['said', 'on', 'June']
words_before_starting_loc
['spokesman', 'Steel</markup>', '<markup>US']
company:- Shares
words_after_ending_loc
['of', '<markup>ThyssenKrupp', 'AG</markup>']
words_before_starting_loc
[]
company:- Index
words_after_ending_loc
['rose', '15', 'percent']
words_before_starting_loc
['DAX', "country's", 'the']
company:- Interfax
words_after_ending_loc
['report', 'that', "it's"]
words_before_starting_loc
['an', 'denied', 'company']
company:- ThyssenKrupp
words_after_ending_loc
['spokesman', 'Klaus', 'Pepperhoff']
words_before_starting_loc
['Severstal</markup>', '<markup>OAO', "Russia's"]
company:- Pepperhoff
words_after_ending_loc
['today', 'denied', 'the']
words_before_start

In [402]:
ngram = ('<markup>Gazprom', 'Lukoil</markup>')
if(len(ngram)>1 and '<markup>' in ngram[0] and '</markup>' in ngram[len(ngram)-1] and
              all((word[0].isupper() or "markup" in word) for word in ngram) and 
              ["<markup>" in word for word in ngram].count(True)==1):
    print "hallo"


hallo


In [403]:
["markup" in word for word in ngram].count(True)==2

True

In [404]:
import pycountry
country_names = []
for country in pycountry.countries:
    country_names.append(country.name)
    if hasattr(country,'common_name'):
        country_names.append(country.common_name)
    if hasattr(country, 'official_name'):
        country_names.append(country.official_name)

country_names.append("America")
country_names.append("Latin America")
country_names.append("Korea")
country_names.append("South Korea")
country_names.append("North Korea")
country_names.append("U.S.")
country_names.append("Russia")
country_names.append("Siberia")

if "Russia" in country_names:
    print "YES"
    
continent_list = ['Asia', 'Africa', 'South America', 'North America', 'Antarctica', 'Europe', 'Australia']

YES


In [405]:
cities = []
with open('world-cities.csv', 'r') as inp:
        for row in csv.reader(inp):
            if(row and row[3] != "" and row[0]!="name"):
                cities.append(row[0])

cities.append('Bombay')
cities.append('New York')
cities.append('Montreal')
cities.append('Frankfurt')
cities.append('Zurich')
if "Compton" in cities:
    print "YES"

us_states = open('us-states.txt').read().split('\n')
ethni_list = open('ethnicities.txt').read().split('\n')

YES


In [406]:
desig = open('designations.txt').read().split('\n')
desig_lower = [x.lower() for x in desig]

print desig_lower

['chief executive officer', 'chief financial officer', 'co-chief executive', 'chief investment officer', 'analyst', 'ceo', 'cio', 'cto', 'cfo', 'manager', 'director', 'president', 'vice-president', 'vice president', 'chief', 'executive', 'officer', 'staff', 'secretary', 'attorney', 'jr', 'sr', 'officials', 'officers', 'analysts', 'managers', 'directors', 'co-chief', 'co chief', 'commissioner', 'minister', 'magistrate', 'judge', 'chairman']


In [407]:
pruned_pos = open('pruned_pos.txt').read().split('\n')
stock_x = open('stock-exchange-list.txt').read().split('\n')
unigram_prune = open('unigram-prune-neg.txt').read().split('\n')
special_char = [';', ',', ':', '?']
prune = open('pruning-neg.txt').read().split('\n') 
prune_lower = [x.lower() for x in prune]
suffixes = open('company-suffixes.txt').read().split('\n')
prefixes = open('company-prefixes.txt').read().split('\n')
rows = 0
print len(processed_list)
for doc in processed_list:
    rows += len(doc)


7966


In [408]:
print pruned_pos
new_proc_list = []
pruned_list = []

for inst in processed_list:
    if inst[0] not in pruned_pos: 
        if inst[0] in country_names or inst[0] in continent_list or inst[0] in cities or inst[0] in stock_x or inst[0] in us_states:
            pruned_list.append(inst)
        elif "'s" in inst[0]:
            pruned_list.append(inst)
        elif inst[0].endswith('n') and inst[0][:-1] in country_names or str in continent_list:
            pruned_list.append(inst)
        elif "based" in inst[0]:
            pruned_list.append(inst)
        elif any(d in inst[0].lower().split() for d in desig_lower):
            pruned_list.append(inst)
        elif any(p in inst[0].lower().split() for p in prune_lower):
            pruned_list.append(inst)
        elif '(' in inst[0] or ')' in inst[0]:
            pruned_list.append(inst)
        elif any(c in inst[0] for c in special_char):
            pruned_list.append(inst)
        elif inst[0].startswith("A ") or inst[0].startswith("An ") or inst[0].startswith("And ") or inst[0].startswith("This"):
             pruned_list.append(inst)
        elif inst[0].endswith("And") or inst[0].endswith("This"):
             pruned_list.append(inst)
        elif len(inst[0].split()) == 1:
            if len(inst[0]) == 1:
                pruned_list.append(inst)
            elif inst[0] in unigram_prune or inst[0] in ethni_list:
                pruned_list.append(inst)
            elif inst[0] == inst[0].upper():
                pruned_list.append(inst)
            elif inst[0] in suffixes or inst[0] in prefixes:
                pruned_list.append(inst)
            else:
                new_proc_list.append(inst)
        else:
            new_proc_list.append(inst)
    else:
        new_proc_list.append(inst)
            
                
print len(pruned_list)
for eg in pruned_list:
    if eg[4] == 1:
        print "HOLY MOLY!"
#writeToCsv(processed_list)
print len(new_proc_list)
print len(pruned_list) + len(new_proc_list)
#print pruned_list



['ACE', "Sam's Club", 'ICBC', 'KKR', 'CSN', 'Tata', 'Wheeling', "River's Edge Pharmaceuticals LLC", "River's Edge", 'RWE', 'Fremont', 'News Corp', 'GM', 'Liberty', 'Financial Services Authority', 'FSA', 'NZOG', 'Norilsk', 'NTP', 'MPC', 'PPR', 'THQ', 'Nokia', 'RCS', 'Intercontinental Exchange Inc', 'EON', 'RHB', 'WCM', 'Macclesfield', 'Carrefour', 'KPI', 'TXU', 'ECI', 'BP', 'TNK-BP', 'Imperial', 'Zug', 'TDF', 'TMK', "Ivernia Inc's", 'SAIC', 'GE', 'PICC', 'Saku', 'Winterthur', 'ZKB', "Rupert Murdoch's News Corp", 'PIK', 'VTB', 'UBS', 'IGT', 'BHP', 'BBH', 'China Banking Regulatory Commission', 'Compton', 'Centennial', 'NPD', 'HSBC', "Victoria's Secret", 'MAN', 'MBIA', 'Freeport', 'MGIC', 'PMI', 'Stillwater', "Dr Reddy's Laboratories Ltd", "Royal Mail's", 'Cia Siderurgica Nacional SA', 'Cia Siderurgica Nacional', 'Vector Hospitality Plc', 'Vector', 'Victory']
3833
HOLY MOLY!
HOLY MOLY!
HOLY MOLY!
HOLY MOLY!
HOLY MOLY!
HOLY MOLY!
HOLY MOLY!
HOLY MOLY!
HOLY MOLY!
HOLY MOLY!
HOLY MOLY!
HOLY M

In [409]:
'''thefile = open('pruned_pos.txt', 'w')
for item in pruned_pos:
    #print item
    thefile.write("%s\n" % item)
len(pruned_pos)
print pruned_list'''

'thefile = open(\'pruned_pos.txt\', \'w\')\nfor item in pruned_pos:\n    #print item\n    thefile.write("%s\n" % item)\nlen(pruned_pos)\nprint pruned_list'

In [410]:
with open("train_pruned.csv", "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    for val in new_proc_list:
        writer.writerow(val)